In [154]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import tree
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [34]:
# df1 = pd.read_csv('all_seasons.csv',index_col='Unnamed: 0')
# #df.isnull().sum()
# df['Name'] = df1['player_name']
# df

df = pd.read_csv('all_seasons.csv',index_col='Unnamed: 0')
df1 = df[['player_name','team_abbreviation','country','age','player_height','player_weight','pts','reb']]
dftarget = df1[df1['age'] <= 25][df1['player_height'] >= 180][df1['player_weight'] <= 90][df1['pts'] >= 6][df1['reb'] >= 3]
dftarget
labeltarget = 1
dftarget['Rekrut'] = labeltarget
dftarget

dfnontarget = df1.drop(dftarget.index)
dfnontarget

labelnontarget = 0
dfnontarget['Rekrut'] = labelnontarget
dfnontarget

df1rekrut = pd.concat([dftarget,dfnontarget])
df1rekrut =  df1rekrut.sort_index(ascending=True)

Z:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


# Standardisasi & Cek Outlier

In [3]:
scaler = StandardScaler()
scalerheight = scaler.fit_transform(df1rekrut[['player_height']])
scalerweight = scaler.fit_transform(df1rekrut[['player_weight']])
scalerpts = scaler.fit_transform(df1rekrut[['pts']])
scalerreb = scaler.fit_transform(df1rekrut[['reb']])
scalerage = scaler.fit_transform(df1rekrut[['age']])
df1rekrut['Z AGE'] = scalerage
df1rekrut['Z HEIGHT'] = scalerheight
df1rekrut['Z WEIGHT'] = scalerweight
df1rekrut['Z PTS'] = scalerpts
df1rekrut['Z REB'] = scalerreb
df1rekrut
dfstd = scaler.fit_transform(df1rekrut[['player_height','player_weight','pts','reb','age']])
dfstd
dfstd = pd.DataFrame(dfstd)
dfstd
dfstd['target'] = df1rekrut['Rekrut']
print(df1rekrut)

            player_name team_abbreviation     country   age  player_height  \
0        Chris Robinson               VAN         USA  23.0         195.58   
1             Matt Fish               MIA         USA  27.0         210.82   
2          Matt Bullard               HOU         USA  30.0         208.28   
3          Marty Conlon               BOS         USA  29.0         210.82   
4       Martin Muursepp               DAL         USA  22.0         205.74   
...                 ...               ...         ...   ...            ...   
9556     Elijah Millsap               PHX         USA  29.0         200.66   
9557      Elfrid Payton               ORL         USA  23.0         193.04   
9558        Edy Tavares               CLE  Cabo Verde  25.0         220.98   
9559  Willy Hernangomez               NYK       Spain  23.0         210.82   
9560    Wilson Chandler               DEN         USA  30.0         203.20   

      player_weight   pts  reb  Rekrut     Z AGE  Z HEIGHT  Z W

In [4]:
print(df1rekrut['Z HEIGHT'].mean())
print(df1rekrut['Z WEIGHT'].mean())
print(df1rekrut['Z PTS'].mean())
print(df1rekrut['Z REB'].mean())
print(df1rekrut['Z AGE'].mean())

print(df1rekrut['Z HEIGHT'].std())
print(df1rekrut['Z WEIGHT'].std())
print(df1rekrut['Z PTS'].std())
print(df1rekrut['Z REB'].std())
print(df1rekrut['Z AGE'].std())

-1.5993638794892286e-15
4.1953273938331177e-16
2.458027506041765e-16
-5.882637634924216e-17
9.30840896349773e-16
1.0000522998875856
1.0000522998875985
1.0000522998875951
1.0000522998875883
1.0000522998876242


In [32]:
dfstd[(dfstd < -2.5) | (dfstd > 2.5)] = np.NaN
dfstd = dfstd.dropna()
dfstd.columns=['AGE','HEIGHT','WEIGHT','PTS','REB','TARGET']
dfstd

,AGE,HEIGHT,WEIGHT,PTS,REB,TARGET
0,-0.578993,-0.802604,-0.589390,-0.747850,-0.996742,0
1,1.063130,0.439607,-1.319614,-1.108712,-0.077272,0
2,0.789443,0.439607,-0.606372,-0.787946,0.612331,0
3,1.063130,0.794524,-0.045967,0.334735,0.382463,0
4,0.515755,0.439607,-0.742228,-0.787946,-1.226609,0
...,...,...,...,...,...,...
9556,-0.031619,-0.270228,-1.115831,-0.226605,0.382463,0
9557,-0.852680,-1.334980,0.803131,0.455022,-0.996742,1
9558,2.157878,1.504359,-0.691282,0.775788,-0.537007,0
9559,1.063130,0.617065,0.021961,1.377224,-0.996742,0


In [6]:
datastd = np.array(dfstd)
datastd
datastd = np.array(dfstd.iloc[:,:-1])
datastd

array([[-0.57899275, -0.80260413, -0.58938997, -0.74785002, -0.99674183],
       [ 1.06312962,  0.43960663, -1.31961423, -1.10871173, -0.07727165],
       [ 0.78944256,  0.43960663, -0.60637193, -0.78794577,  0.61233098],
       ...,
       [ 2.15787786,  1.50435871, -0.69128173,  0.7757883 , -0.53700674],
       [ 1.06312962,  0.61706531,  0.02196056,  1.37722448, -0.99674183],
       [ 0.24206844,  0.08468927,  1.29560752,  1.17674575,  0.61233098]])

# SPLITTING DATASET

In [20]:
x = datastd
y = dfstd['target']

In [8]:
xtr, xts, ytr, yts = train_test_split(
    x, y, test_size = 0.18
)
print(xtr.shape)
print(xts.shape)
print(ytr.shape)
print(yts.shape)

(7337, 5)
(1611, 5)
(7337,)
(1611,)


# 1. K-NEAREST NEIGHTBORS

#### K Value

In [9]:
len(xtr)
k = round(len(xtr)**0.5)
# k = 86
k = k+1
k

87

In [80]:
modelK = KNeighborsClassifier(n_neighbors = k)
modelK.fit(xtr,ytr)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=87, p=2,
                     weights='uniform')

In [81]:
print(modelK.score(xts,yts))

0.9883772910147519


In [113]:
# urutannya umur, height, weight, pts, reb
print(modelK.predict(dfstd[['AGE','HEIGHT','WEIGHT','PTS','REB']]))
#dfstd['TARGET_PREDICT_KNEAREST'] = modelK.predict(dfstd[['AGE','HEIGHT','WEIGHT','PTS','REB']])

PrediksiKNEAR = modelK.predict(xts)



[0 0 0 ... 0 0 0]


### BALANCED ACCURACY, PRECISION, RECALL, F1, ROC_AUC SCORE

In [129]:
print('BACC =',balanced_accuracy_score(yts,PrediksiKNEAR))
print('Precision+ =', precision_score(yts, PrediksiKNEAR, pos_label=1))
print('Precision- =', precision_score(yts, PrediksiKNEAR, pos_label=0))
print('Recall+ =', recall_score(yts, PrediksiKNEAR, pos_label=1))
print('Recall- =', recall_score(yts, PrediksiKNEAR, pos_label=0))
print('F1 score+ =', f1_score(yts, PrediksiKNEAR, pos_label=1))
print('F1 score- =', f1_score(yts, PrediksiKNEAR, pos_label=0))
print('Roc Auc Score =',roc_auc_score(yts, PrediksiKNEAR))

BACC = 0.6829268292682926
Precision+ = 1.0
Precision- = 0.9882988298829883
Recall+ = 0.36585365853658536
Recall- = 1.0
F1 score+ = 0.5357142857142857
F1 score- = 0.9941149841557265
Roc Auc Score = 0.6829268292682926


# 2. LOGISTIC REGRESSION

### TUNING PARAMETER

In [37]:
model = LogisticRegression()
model.fit(dfstd[['AGE','HEIGHT','WEIGHT','PTS','REB']],dfstd['TARGET'])
model.score(dfstd[['AGE','HEIGHT','WEIGHT','PTS','REB']],dfstd['TARGET'])

0.9854716137684398

In [39]:
xtr,xts,ytr,yts = train_test_split(dfstd[['AGE','HEIGHT','WEIGHT','PTS','REB']],dfstd['TARGET'])
model = LogisticRegression()
model.fit(xtr,ytr)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
model.score(xtr,ytr)

0.9848010728654448

In [42]:
print(cross_val_score(model,xtr,ytr,cv=5))
print(np.mean(cross_val_score(model,xtr,ytr)))

[0.98436337 0.9880775  0.98211624 0.98882265 0.98137109]
0.9849501693940985


In [172]:
penalty = ['l2','l2','elasticnet','none']
solver = ['lbfgs','liblinear','sag','saga']
max_iter = [1,10,100,1000,10000]
param = {'penalty':penalty,'solver':solver,'max_iter':max_iter}
param

{'penalty': ['l2', 'l2', 'elasticnet', 'none'],
 'solver': ['lbfgs', 'liblinear', 'sag', 'saga'],
 'max_iter': [1, 10, 100, 1000, 10000]}

In [173]:
model = LogisticRegression()
modelgs = GridSearchCV(model,param,cv=5)
modelgs.fit(xtr,ytr)
modelgs.best_params_

Z:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
Z:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_ms

{'max_iter': 10, 'penalty': 'l2', 'solver': 'sag'}

In [48]:
modeltuning = LogisticRegression(
    max_iter=10,penalty='l2',solver='sag')
modeltuning.fit(xtr,ytr)
print(modeltuning.score(xts,yts))

0.9848010728654448


Z:\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [137]:
PrediksiLOGISTIC = modeltuning.predict(xts)
print(modeltuning.score(xts,yts))

0.9848010728654448


In [131]:
print('BACC =',balanced_accuracy_score(yts,PrediksiLOGISTIC))
print('Precision+ =', precision_score(yts, PrediksiLOGISTIC, pos_label=1))
print('Precision- =', precision_score(yts, PrediksiLOGISTIC, pos_label=0))
print('Recall+ =', recall_score(yts, PrediksiLOGISTIC, pos_label=1))
print('Recall- =', recall_score(yts, PrediksiLOGISTIC, pos_label=0))
print('F1 score+ =', f1_score(yts, PrediksiLOGISTIC, pos_label=1))
print('F1 score- =', f1_score(yts, PrediksiLOGISTIC, pos_label=0))
print('Roc Auc Score =',roc_auc_score(yts, PrediksiLOGISTIC))

BACC = 0.7050402061397663
Precision+ = 0.6296296296296297
Precision- = 0.9891402714932127
Recall+ = 0.4146341463414634
Recall- = 0.9954462659380692
F1 score+ = 0.5
F1 score- = 0.9922832501134816
Roc Auc Score = 0.7050402061397661


# TREE DECISION

In [152]:
modelT = tree.DecisionTreeClassifier()
#modelT.fit(dfstd[['AGE','HEIGHT','WEIGHT','PTS','REB']],dfstd['TARGET'])
modelT.fit(xtr,ytr)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [161]:
criterion = ['gini','entropy']
splitter = ['best','random']

paramT = {'criterion':criterion, 'splitter':splitter}
paramT

modelT = tree.DecisionTreeClassifier()
modelT = modelT.fit(xtr,ytr)
modelT = RandomizedSearchCV(estimator=modelT, param_distributions=paramT, cv=5)
modelT.fit(xtr, ytr)
modelT.best_params_

Z:\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:281: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


{'splitter': 'best', 'criterion': 'gini'}

In [165]:
modelBaruT = tree.DecisionTreeClassifier(splitter='best',criterion='gini')
modelBaruT.fit(xtr, ytr)
predictT = modelBaruT.predict(xts)
print(modelBaruT.score(xts, yts))

1.0


In [166]:
PrediksiTREE = modeltuning.predict(xts)

In [167]:
print('BACC =',balanced_accuracy_score(yts,PrediksiTREE))
print('Precision+ =', precision_score(yts, PrediksiTREE, pos_label=1))
print('Precision- =', precision_score(yts, PrediksiTREE, pos_label=0))
print('Recall+ =', recall_score(yts, PrediksiTREE, pos_label=1))
print('Recall- =', recall_score(yts, PrediksiTREE, pos_label=0))
print('F1 score+ =', f1_score(yts, PrediksiTREE, pos_label=1))
print('F1 score- =', f1_score(yts, PrediksiTREE, pos_label=0))
print('Roc Auc Score =',roc_auc_score(yts, PrediksiTREE))

BACC = 0.7050402061397663
Precision+ = 0.6296296296296297
Precision- = 0.9891402714932127
Recall+ = 0.4146341463414634
Recall- = 0.9954462659380692
F1 score+ = 0.5
F1 score- = 0.9922832501134816
Roc Auc Score = 0.7050402061397661


In [168]:
print(modelK.score(xts,yts))
print(modeltuning.score(xts,yts))
print(modelT.score(xts,yts))

0.9883772910147519
0.9848010728654448
1.0


In [169]:
# Tertinggi Model Decision Tree

In [171]:
dfindo = pd.read_csv('pemainindotest.csv')
dfindo
#dfTest = dfTest.drop(['Club','Country'], axis=1)
# nilaiX = dfTest.iloc[:,1:]
# dfTest['Target'] = modelBaruDT.predict(nilaiX)
x = dfindo.iloc[:,3:]
x
dfindo['PREDICT'] = modelBaruT.predict(x)
dfindo

,Name,Club,Country,Age,Height,Weight,pts,reb,PREDICT
0,Andakara Prastawa Dyaksa,Pelita Jaya Bakrie,Indonesia,24,190,90,7,6,0
1,Reggie Mononimbar,Pelita Jaya Bakrie,Indonesia,21,185,86,6,3,0
2,Hardianus Lakudu,Satria Muda Pertamina Jakarta,Indonesia,23,178,83,10,3,0
3,Kevin Yonas Sitorus,Satria Muda Pertamina Jakarta,Indonesia,26,185,75,11,4,0
4,Arki Dikania Wisnu,Satria Muda Pertamina Jakarta,Indonesia,20,183,80,5,2,0
5,Laurentius Steven Oei,Satria Muda Pertamina Jakarta,Indonesia,21,191,85,4,10,0
6,Mei Joni,Stapac,Indonesia,25,188,90,7,5,0
7,Vincent Rivaldi Kosasih,Stapac,Indonesia,23,179,87,1,2,0
8,Hardian Wicaksono,Pacific Caesar Surabaya,Indonesia,21,177,80,9,8,0
9,Brandon Jawato,Louvre Surabaya,Indonesia,24,182,85,6,5,0
